# Modelos de Regressão: Regressão Linear Múltipla

### Importando libs e funções:

Importando libs

In [0]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from matplotlib.colors import ListedColormap

Importando funções

In [0]:
# Função de escalonamento
def feature_scaling(data):
    sc = StandardScaler()
    return sc.fit_transform(data)

# Função que gera o gráfico dos resultados de regressão
def plot_results_linear(X, y, regressor, title):
    plt.scatter(X, y, color = 'red')
    plt.plot(X, regressor.predict(X), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()

# Função que gera o gráfico dos resultados de regerssão polinomial
def plot_results_poly(X, y, lin_reg_poly, poly_reg, title):
    plt.scatter(X, y, color = 'red')
    plt.plot(X, lin_reg_poly.predict(poly_reg.fit_transform(X)), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()    
    
# Função que gera o gráfico dos resultados de arvores
def plot_results_reg(X, y, regressor, title):     
    X_grid = np.arange(min(X), max(X), 0.01)
    X_grid = X_grid.reshape((len(X_grid), 1))
    plt.scatter(X, y, color = 'red')
    plt.plot(X_grid, regressor.predict(X_grid), color = 'blue')
    plt.title(title)
    plt.xlabel('Tamanho do Lote')
    plt.ylabel('Preço de Vendas')
    plt.show()
     

### Etapa de exploração e tratamento dos **dados**

Importando o dataset do nosso estudo. O objetivo dos modelos de regressão será de predizer o preço das casas de acordo com diferentes caracteristicas como: localização, área, etc.

Fonte: [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/r4phael/ml-course/master/data/pricing_houses.csv')

#Selecionando uma amostragem dos dados para uma melhor visualização
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'OverallCond','YearBuilt', 'MSZoning', 'SalePrice']]

Descrevendo o dataset

In [236]:
df.describe()

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,2.758904,472.980137,5.575342,1971.267808,180921.195890
std,9981.264932,40.177307,213.804841,1.112799,30.202904,79442.502883
min,1300.000000,0.000000,0.000000,1.000000,1872.000000,34900.000000
25%,7553.500000,0.000000,334.500000,5.000000,1954.000000,129975.000000
50%,9478.500000,0.000000,480.000000,5.000000,1973.000000,163000.000000
75%,11601.500000,0.000000,576.000000,6.000000,2000.000000,214000.000000
max,215245.000000,738.000000,1418.000000,9.000000,2010.000000,755000.000000


Visualizando o dataset

In [237]:
df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,MSZoning,SalePrice
0,8450,0,548,5,2003,RL,208500
1,9600,0,460,8,1976,RL,181500
2,11250,0,608,5,2001,RL,223500
3,9550,0,642,5,1915,RL,140000
4,14260,0,836,5,2000,RL,250000


Preenchendo os valores númericos nulos (NA) com a mediana.

In [238]:
df = df.fillna(df.median())

df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,MSZoning,SalePrice
0,8450,0,548,5,2003,RL,208500
1,9600,0,460,8,1976,RL,181500
2,11250,0,608,5,2001,RL,223500
3,9550,0,642,5,1915,RL,140000
4,14260,0,836,5,2000,RL,250000


Definindo as variáveis independentes

In [239]:
X = df.loc[:, 'LotArea'].values.reshape(-1,1)
X[:5]


array([[ 8450],
       [ 9600],
       [11250],
       [ 9550],
       [14260]])

Definindo as variáveis dependentes.

In [240]:
y = df.loc[:, 'SalePrice'].values.reshape(-1,1)
y[:5]


array([[208500],
       [181500],
       [223500],
       [140000],
       [250000]])

###  Etapa de Classificação e Validação do Modelo

Selecionando as principais features do dataset:

In [241]:
df = df.loc[:, ['LotArea', 'PoolArea', 'GarageArea', 'OverallCond','YearBuilt', 'MSZoning', 'SalePrice']].reindex()

df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,MSZoning,SalePrice
0,8450,0,548,5,2003,RL,208500
1,9600,0,460,8,1976,RL,181500
2,11250,0,608,5,2001,RL,223500
3,9550,0,642,5,1915,RL,140000
4,14260,0,836,5,2000,RL,250000


Exibir os valores distintos da coluna MSZoning. **Segue legenda:**
FV-Residencial de Vila Flutuante, RH- Residencial de Densidade Alta, RL-Residencial de Densidade Baixa, RM-Residencial de Densidade Média.

In [242]:
df['MSZoning'].unique()

array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object)

Codificando as variáveis Categóricos e evitando a armadilha da variável Dummy. **Obs:** Algumas técnicas de aprendizado de máquina exigem que você elimine uma dimensão da apresentação para evitar a dependência entre as variáveis. Isso pode ser feito através do parâmetro "drop_first = True".

In [243]:
df = pd.get_dummies(df , columns = ['MSZoning'], drop_first=True)

df.head(5)

,LotArea,PoolArea,GarageArea,OverallCond,YearBuilt,SalePrice,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
0,8450,0,548,5,2003,208500,0,0,1,0
1,9600,0,460,8,1976,181500,0,0,1,0
2,11250,0,608,5,2001,223500,0,0,1,0
3,9550,0,642,5,1915,140000,0,0,1,0
4,14260,0,836,5,2000,250000,0,0,1,0


Definindo as variáveis indepedentes e dependentes e dividindo o dataset em conjunto de treinamento e testes:

In [0]:
X = df[df.columns[~df.columns.isin(['SalePrice'])]].values
y = df['SalePrice'].values.reshape(-1,1)

# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Normalizando as features 

In [245]:
X_train = feature_scaling(X_train)
X_test = feature_scaling(X_test)

X_train[:5]

array([[-0.21289571, -0.07099284, -0.86383727,  0.3722173 , -0.45546896,
        -0.21802209, -0.11405937,  0.5138767 , -0.41556078],
       [-0.26524463, -0.07099284, -0.45626397,  1.26860866,  0.71860895,
        -0.21802209, -0.11405937,  0.5138767 , -0.41556078],
       [-0.17784146, -0.07099284, -2.25716927,  1.26860866, -1.98829291,
        -0.21802209, -0.11405937,  0.5138767 , -0.41556078],
       [-0.32447422, -0.07099284, -1.11975539,  1.26860866, -1.10773447,
        -0.21802209, -0.11405937,  0.5138767 , -0.41556078],
       [-0.52903482, -0.07099284, -0.79748813,  0.3722173 , -1.53170705,
        -0.21802209, -0.11405937,  0.5138767 , -0.41556078]])

Importando e treinando o modelo de Regressao Linear Multipla com o Conjunto de Treinamento:

In [246]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Prevendo os resultados do modelo criado com o conjunto de testes

In [247]:
y_pred = regressor.predict(X_test)

y_pred[:5]

array([[162102.94940288],
       [241746.81922749],
       [119733.72469521],
       [147800.757112  ],
       [285853.71056469]])

Avaliando o modelo com a métrica r2

In [248]:
regressor.score(X_test, y_test)

0.5043465841755175